# エントロピー符号化

次のような8種類の文字からなる文字列をデジタル信号であらわすことを考える．

In [1]:
signal = 'a' * 16 + 'b' * 4 + 'c' * 4 + 'd' * 2 + 'e' * 2 + 'f' * 2 + 'g' * 1 + 'h' * 1

In [2]:
signal

'aaaaaaaaaaaaaaaabbbbccccddeeffgh'

## 固定長符号

固定長符号を用いて文字列に含まれる文字に符号を割り当てる関数`fixed_length_code_table`を定義する．

In [3]:
def fixed_length_code_table(signal):
    """Assign a code to a symbol using fixed length coding."""
    table = {}
    
    for i, symbol in enumerate(set(signal)):
        table[symbol] = bin(i)[2:]
    bit = len(max(table.values()))
    
    return {symbol: code.zfill(bit) for symbol, code in table.items()}

定義した関数`fixed_length_code_table`を実行すると，固定長符号の符号表を返す．

In [4]:
fixed_length_code_table(signal)

{'h': '000',
 'c': '001',
 'a': '010',
 'g': '011',
 'e': '100',
 'f': '101',
 'd': '110',
 'b': '111'}

## ハフマン符号

文字列に含まれる文字の出現頻度を調べる関数`counter`を定義する（モジュール`collections`のクラス`Counter`を用いてもよい）．

In [5]:
def counter(signal):
    """Count symbols in the signal."""
    count = {}
    for symbol in signal:
        try:
            count[symbol] += 1
        except KeyError:
            count[symbol] = 1
    
    return count

ハフマン符号を用いて文字列に含まれる文字に符号を割り当てる関数`huffman_code_table`を定義する．

In [6]:
def huffman_code_table(signal):
    """Assign a code to a symbol using Huffman coding."""
    count = list(counter(signal).items())
    table = {symbol: '' for symbol, _ in count}

    while len(count) > 1:
        count.sort(key=lambda t: t[1])

        tree1 = count.pop(0)
        tree2 = count.pop(0)

        for symbol in tree1[0]:
            table[symbol] += '0'
        for symbol in tree2[0]:
            table[symbol] += '1'

        count.append((tree1[0] + tree2[0], tree1[1] + tree2[1]))
    
    if len(table) == 1:
        return {symbol: '0' for symbol in table}
    else:
        return {symbol: code[::-1] for symbol, code in table.items()}

定義した関数`huffman_code_table`を実行すると，ハフマン符号の符号表を返す．

In [7]:
huffman_code_table(signal)

{'a': '0',
 'b': '100',
 'c': '101',
 'd': '1100',
 'e': '1101',
 'f': '1110',
 'g': '11110',
 'h': '11111'}

## エントロピーと平均符号長

文字列のエントロピーを求める関数`entropy`を定義する．

In [8]:
from math import log2


def entropy(signal):
    """Compute Entropy of the signal."""
    count = counter(signal)
    length = len(signal)
    
    return -sum(c * log2(c / length) for c in count.values()) / length

定義した関数`entropy`を実行すると，文字列のエントロピーを計算する．

In [9]:
entropy(signal)

2.3125

文字列に含まれる文字に符号を割り当てた場合の平均符号長を求める関数`average_code_length`を定義する．

In [10]:
def average_code_length(signal, table):
    """Coumpte average code length."""
    count = counter(signal)
    
    return sum(c * len(table[s]) for s, c in count.items()) / len(signal)

定義した関数`average_code_length`を実行すると，文字列を符号化した際の平均符号長を計算する．

固定長符号の平均符号長

In [11]:
average_code_length(signal, fixed_length_code_table(signal))

3.0

ハフマン符号の平均符号長

In [12]:
average_code_length(signal, huffman_code_table(signal))

2.3125

ハフマン符号の場合，平均符号長とエントロピーが一致する．

## エンコードとデコード

符号表を参照して文字列をエンコードする関数`encode`を定義する．

In [13]:
def encode(signal, table):
    """Encode the signal using code table."""
    
    return ''.join(table[symbol] for symbol in signal)

符号表を参照して符号列をデコードする関数`decode`を定義する．

In [14]:
def decode(signal, table):
    """Decode the signal using code table."""
    table = {code: symbol for symbol, code in table.items()}

    code = ''
    decoded = ''
    for binary in signal:
        code += binary
        if code in table:
            decoded += table[code]
            code = ''
        
    return decoded

ハフマン符号でエンコードした文字列

In [15]:
table = huffman_code_table(signal)

digital_signal = encode(signal, table)

In [16]:
digital_signal

'00000000000000001001001001001011011011011100110011011101111011101111011111'

符号化した文字列をデコードすると元の文字列が得られる．

In [17]:
decode(digital_signal, table)

'aaaaaaaaaaaaaaaabbbbccccddeeffgh'